In [531]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [532]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [533]:
from src.data.data_loader import load_all_data

df_raw = load_all_data()

In [534]:
df_raw = df_raw.reset_index(drop=True)

In [540]:
df_raw = df_raw.drop_duplicates(subset=['func_documentation'])

In [572]:
import numpy as np
X = np.stack(df_raw["g_ave_reps"])

In [640]:
from sklearn.cluster import DBSCAN

# cluster each parent cluster

parent = DBSCAN(eps=0.5, min_samples=3)

parent_labels = parent.fit_predict(X)

In [641]:
import pandas as pd
df = pd.DataFrame(X)
df['label'] = parent_labels
#df = df.reset_index()
clusters = df.groupby('label', as_index=False, group_keys=True)
#groups = {k: v.values for k, v in df.groupby('label')}
#print(groups)

In [642]:
def cluster_sub_clusters(cluster):
    if cluster.name == -1:
        print("Skipping noise cluster")
        cluster["sub_label"] = -1
        return cluster

    #if cluster.name == -1:
    #    print("Skipping noise cluster")
    #    return
    
    algorithm = DBSCAN(eps=2, min_samples=5)
    sub_clusters = algorithm.fit(cluster.drop('label', axis=1).to_numpy())
    cluster["sub_label"] = sub_clusters.labels_.tolist()
    return cluster

clusters = clusters.apply(cluster_sub_clusters)


Skipping noise cluster


In [643]:
clusters = clusters.droplevel(0).sort_index()

In [644]:
clusters.label.value_counts().shape

(40,)

In [645]:
clusters

,0,1,2,3,4,5,6,7,8,9,...,4088,4089,4090,4091,4092,4093,4094,4095,label,sub_label
0,0.352719,0.830436,0.033339,0.237478,0.399952,0.796021,-0.202346,-1.081397,0.064935,-0.306452,...,0.554941,0.505535,-0.805905,0.668801,0.307411,0.240458,0.321426,0.653370,-1,-1
1,0.315083,0.793894,-0.037793,0.256857,0.513136,0.964972,-0.121371,-1.093389,0.159361,-0.408715,...,0.561411,0.762463,-0.923000,0.882743,0.379599,0.299347,0.251709,0.465588,-1,-1
2,0.805990,0.969580,0.751512,-1.068722,-0.222679,-0.271944,-0.180482,-0.122487,0.374881,0.679594,...,-0.314225,0.608536,-0.070194,0.025360,-0.354162,-0.034466,0.654991,-0.194940,-1,-1
3,0.260986,0.447498,0.605103,-0.175920,-0.573184,0.281979,0.185635,-0.176148,0.163705,0.087285,...,-0.210508,0.171072,0.293322,0.744013,0.188687,0.717073,-0.148546,-0.457200,-1,-1
4,0.344960,0.092580,0.793630,0.221295,-0.130600,0.450217,-0.398687,-0.225323,0.192957,0.543535,...,0.682055,0.240797,0.278898,0.183188,0.048695,0.735935,-0.120839,0.085078,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4469,0.050781,0.604869,0.066492,-0.326941,-0.243736,-0.170143,-0.556178,0.751576,-0.158600,0.417670,...,0.209695,0.577196,-0.572833,-0.221424,0.325103,0.233749,1.000910,0.910149,-1,-1
4470,0.373157,0.499825,0.131793,-0.077973,-0.072598,0.283464,-0.060214,1.257658,0.624086,0.227925,...,0.057187,0.340962,-0.232556,0.141942,0.611533,0.535476,0.522013,0.226693,-1,-1
4471,0.455558,-0.084019,0.070801,0.619380,-0.206617,-0.248978,-0.429772,0.784585,0.466005,0.455027,...,0.145870,1.011884,0.477956,0.126531,0.156246,0.172393,0.104659,-0.307862,-1,-1
4472,0.145493,0.389114,0.994847,0.166832,-0.249153,-0.207544,-0.370391,0.203541,-0.091527,0.218300,...,0.715083,-0.460935,-0.151195,0.100507,-0.167227,-0.263388,0.098550,0.034390,-1,-1


In [646]:
clusters["sub_label"].value_counts()

-1    4437
 0      37
Name: sub_label, dtype: int64

In [647]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

smoothie = SmoothingFunction().method1

def calc_bleu_score(target, generated):
    #reference = target.strip(" ").splitlines()
    reference = target.strip().splitlines()
    #reference = list(filter(None, reference))
    #reference = list(map(str.strip, reference))
    #hypothesis = generated.strip(" ").splitlines()
    hypothesis = generated.strip().splitlines()
    #hypothesis = list(filter(None, hypothesis))
    #hypothesis = list(map(str.strip, hypothesis))
    list_of_references = [[line.split()] for line in reference]
    hypotheses = [line.split() for line in hypothesis]
    if len(list_of_references) < len(hypotheses):
        diff = len(hypotheses) - len(list_of_references)
        list_of_references.extend([[[]]] * diff)
    elif len(list_of_references) > len(hypotheses):
        diff =  len(list_of_references) - len(hypotheses)
        hypotheses.extend([[]] * diff)
    try:
        return corpus_bleu(list_of_references, hypotheses)#, smoothing_function=smoothie,)
    except Exception as e:
        return -1#pd.NA

In [648]:
df_raw["bleu_score"] = df_raw.apply(lambda row: calc_bleu_score(row["func_code"], row["generated"]), axis=1)

/Users/andrestorhaug/Code/Projects/code-comment-prompt-engineering/.venv/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/andrestorhaug/Code/Projects/code-comment-prompt-engineering/.venv/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/andrestorhaug/Code/Projects/code-comment-prompt-engineering/.venv/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Theref

In [649]:
import numpy as np
import random

vectors = clusters.drop(['label', 'sub_label'], axis=1).to_numpy()

metadata = np.stack(
    [
        df_raw["func_documentation"].apply(lambda x: x.encode("unicode_escape").decode("utf-8")).values,
        df_raw["func_code"].apply(lambda x: x.encode("unicode_escape").decode("utf-8")).values,
        df_raw['bleu_score'].values.astype(str),
        clusters['label'].values.astype(str),
        clusters['sub_label'].values.astype(str),
    ],
    axis=1
).tolist()


In [650]:
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%b-%d_%H-%M-%Ss")

writer = SummaryWriter(log_dir='../.data/runs/' + dt_string + '-ave')
writer.add_embedding(mat=vectors, metadata=metadata, metadata_header=['func_documentation', 'func_code', 'bleu_score', 'cluster', 'sub_cluster'])
writer.close()

In [651]:
df_raw["cluster"] = clusters["label"]
df_raw["sub_cluster"] = clusters["sub_label"]

/var/folders/bg/v12j1gk96gz44lcngd9z6ddh0000gn/T/ipykernel_45789/1047115690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["cluster"] = clusters["label"]
/var/folders/bg/v12j1gk96gz44lcngd9z6ddh0000gn/T/ipykernel_45789/1047115690.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["sub_cluster"] = clusters["sub_label"]


In [652]:
df_raw

,Unnamed: 0.1,Unnamed: 0,contract_name,file_path,contract_address,language,class_name,class_code,class_documentation,class_documentation_type,...,bleu_score,mode,truncated,duration,g_ave_reps,g_max_reps,o_ave_reps,o_max_reps,cluster,sub_cluster
0,0,422,AANG,AANG.sol,0xd655c864ddb738062819d8864951f77251ff8910,Solidity,SafeMath,library SafeMath {\r\n /**\r\n * @dev Re...,/**\r\n * @dev Wrappers over Solidity's arithm...,NatSpecMultiLine,...,8.941232e-01,COMMENT,False,2.554811,"[0.3527191, 0.83043647, 0.033339318, 0.2374780...","[3.3779757, 4.457412, 2.8460157, 3.0819604, 3....","[0.33097878, 0.69757885, 0.100496486, 0.028003...","[2.7769904, 4.218847, 2.2692716, 3.6166346, 3....",-1.0,-1.0
1,1,7975,FFF,contracts/FFF.sol,0xc16b2419494ae0604432297d40cdf0e8d68de8d6,Solidity,SafeMath,library SafeMath {\n /**\n * @dev Retur...,NaN,NaN,...,8.941232e-01,COMMENT,False,2.272884,"[0.31508255, 0.7938935, -0.03779306, 0.2568571...","[3.1013258, 4.5671663, 2.4599104, 3.0501862, 3...","[0.39913902, 0.64368975, 0.0765201, 0.0693836,...","[2.7202299, 4.0962443, 2.1909215, 3.303579, 3....",-1.0,-1.0
2,2,9588,APES,APES.sol,0xee1780df4f7018523628133008a7ca70da4967b8,Solidity,ERC20,contract ERC20 is Context{\r\n using SafeMa...,/**\r\n * @dev Implementation of the {IERC20} ...,NatSpecMultiLine,...,7.071068e-01,COMMENT,False,1.116251,"[0.8059902, 0.9695797, 0.7515121, -1.0687215, ...","[2.3151317, 1.9920294, 3.812863, 1.3659133, 2....","[0.837208, 0.90664333, 0.5939182, -1.0230372, ...","[2.3461704, 1.6422517, 3.0632608, 1.3659133, 2...",-1.0,-1.0
3,3,9394,WHCToken,WHCToken.sol,0xdde5b33a56f3f1c22e5a6bd8429e6ad508bff24e,Solidity,Ownable,contract Ownable is Context {\r\n address p...,NaN,NaN,...,8.034284e-01,COMMENT,False,0.901411,"[0.2609864, 0.44749808, 0.6051026, -0.17592022...","[1.3375204, 2.2761745, 1.4504807, 1.1242895, 1...","[0.31139493, 0.67871684, 0.58714485, -0.311943...","[1.602369, 2.2761745, 2.19575, 0.9518931, 1.21...",-1.0,-1.0
4,4,9635,Flasher,contracts/Controller.sol,0x54f72175c40c3f4857b18fe539a85ddb92f6d985,Solidity,Controller,contract Controller is Ownable {\n using Safe...,NaN,NaN,...,2.113938e-156,COMMENT,False,6.270642,"[0.34495994, 0.09258004, 0.79363036, 0.2212945...","[5.0464034, 3.3600085, 3.9774451, 2.2968931, 2...","[-0.27931824, 0.35097224, 0.84666604, 0.277840...","[4.8049173, 4.336422, 4.989438, 4.266009, 6.80...",-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9857,1167,1376,SafeTransfer,SafeTransfer.sol,0xfc940eee87b66b7eb93bb344ac7b4a3d73571274,Solidity,SafeTransfer,contract SafeTransfer is AccessControl {\r\n ...,NaN,NaN,...,6.903557e-232,COMMENT,False,8.014673,"[0.050780717, 0.6048694, 0.06649177, -0.326940...","[3.4033968, 3.7665024, 3.433598, 3.7552173, 2....","[0.1394893, 0.41799384, 1.0440961, 0.086190365...","[2.1141138, 1.7580241, 3.0663548, 3.591261, 1....",NaN,NaN
9858,1168,1808,MetaCore,MetaCore.sol,0xc381bb219830ab298e3fec16530c21fa0cdd23d1,Solidity,MetaCore,"contract MetaCore is ERC20Interface, Owned {\r...",// -------------------------------------------...,LineComment,...,0.000000e+00,COMMENT,False,12.509467,"[0.37315723, 0.49982506, 0.13179287, -0.077972...","[2.9074082, 3.502232, 4.4562664, 2.9519095, 3....","[-0.09187754, 0.28645805, 0.7776948, -0.184595...","[2.7776146, 2.9931633, 3.6754196, 3.8763802, 4...",NaN,NaN
9865,1175,1235,DolomiteDirectV1,DolomiteDirectV1.sol,0x805f307b960136c82101ae078df78efb08caee2b,Solidity,DolomiteDirectV1,"contract DolomiteDirectV1 is Requestable, IVer...",/**\r\n * @title DolomiteDirectV1\r\n * @autho...,NatSpecMultiLine,...,0.000000e+00,COMMENT,False,9.401768,"[0.455558, -0.08401886, 0.07080086, 0.61938024...","[3.734488, 3.3401568, 2.475507, 3.878598, 2.12...","[0.42435429, 0.23082685, 0.5299888, 0.5994939,...","[3.3189187, 1.8108466, 3.700305, 4.0422235, 2....",NaN,NaN
9875,1185,2370,BinaryOption,BinaryOption.sol,0x645f7dd67479663ee7a42fefec2e55a857cb1833,Solidity,BinaryOptionMarketManager,"contract BinaryOptionM

In [529]:
df_raw.to_pickle("generated_maximum.pkl")